In [109]:
import pandas as pd
import ast
import re

from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

import plotly.graph_objects as go

The dessi dataset can be downloaded from   
https://www.kaggle.com/datasets/sensitivedetection/dessi-dataset-for-structured-sensitive-information  
where the only the DeSSI_v2 folder is necessary

# Evaluation - Presidio Confusion Matrix on Dessi Data

Use a Presidio Confusion Matrx to find label errors for Dessi Data  
Use all Predictions of Presidio without any thresholds so that no wrong predictions are ignored

In [110]:
dessi_train_labels = pd.read_csv("DeSSI_v2/train_labels.csv")
dessi_train = pd.read_csv("DeSSI_v2/train.csv")

dessi_dev_labels = pd.read_csv("DeSSI_v2/dev_labels.csv")
dessi_dev = pd.read_csv("DeSSI_v2/dev.csv")

dessi_test_labels = pd.read_csv("DeSSI_v2/test_labels.csv")
dessi_test = pd.read_csv("DeSSI_v2/test.csv")

dessi_all = pd.concat([dessi_train, dessi_dev, dessi_test], axis=1)
dataset_split = ["train"] * dessi_train.shape[1] + ["validation"] * dessi_dev.shape[1] + ["test"] * dessi_test.shape[1]
dataset_split_df = pd.DataFrame(dataset_split).T
dataset_split_df.columns = dessi_all.columns
dessi_all = pd.concat([dessi_all, dataset_split_df]).reset_index(drop=True)

dessi_all_labels = pd.concat([dessi_train_labels, dessi_dev_labels, dessi_test_labels], axis=0).reset_index(drop=True)
dataset_split_df = pd.DataFrame(dataset_split)
dataset_split_df.columns = ["split_type"]
dessi_all_labels = pd.concat([dessi_all_labels, dataset_split_df], axis=1).reset_index(drop=True)

C:\Users\Luca\AppData\Local\Temp\ipykernel_14688\3396044622.py:2: DtypeWarning:

Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,1040

In [111]:
def read_files(method):
    train = pd.read_csv(f"../../Presidio/old_predictions/dessi_results/train/results_{method}.csv")
    val = pd.read_csv(f"../../Presidio/old_predictions/dessi_results/val/results_{method}.csv")
    test = pd.read_csv(f"../../Presidio/old_predictions/dessi_results/test/results_{method}.csv")
    return train, val, test

def concat_results(train, val, test):
    all_results = pd.concat([train, val, test], axis=1)
    dataset_split = ["train"] * dessi_train.shape[1] + ["validation"] * dessi_dev.shape[1] + ["test"] * dessi_test.shape[1]
    dataset_split_df = pd.DataFrame(dataset_split).T
    dataset_split_df.columns = all_results.columns
    return pd.concat([all_results, dataset_split_df]).reset_index(drop=True).T.reset_index(drop=True).T     #reset index and column names

results_method3_train, results_method3_val, results_method3_test = read_files("columnwise")

results_method_3_all = concat_results(results_method3_train, results_method3_val, results_method3_test)

The Classes of Presidio must be mapped to the classes of dessi to compare both

In [112]:
# Mapping dictionary
# IN_VEHICLE_REGISTRATION, IP_ADDRESS, MEDICAL_LICENSE, URL, US_BANK_NUMBER are matched to "Other_data"
CATEGORY_MAP = {
    "AU_TFN": "NIN",
    "AU_MEDICARE": "NIN",
    "IN_AADHAAR": "NIN",
    "IN_PAN": "NIN",
    "IN_PASSPORT": "Passport",
    "UK_NHS": "NIN",
    "US_ITIN": "NIN",
    "US_SSN": "NIN",
    "AU_ABN": "Organization",
    "AU_ACN": "Organization",
    "CREDIT_CARD": "CCN",
    "DATE_TIME": "Date",
    "shared DATE_TIME": "Date",
    "EMAIL_ADDRESS": "Email",
    "IBAN_CODE": "IBAN",
    "LOCATION": ["Geolocation", "Address", "GPE"],
    "shared LOCATION": ["Geolocation", "Address", "GPE"],
    "NRP": ["Nationality", "Religion"],  
    "shared NRP": ["Nationality", "Religion"],
    "PERSON": "Person",
    "shared PERSON": "Person",
    "PHONE_NUMBER": "Phone_number",
    "shared PHONE_NUMBER": "Phone_number",
    "US_DRIVER_LICENSE": "ID_Card",
    "shared US_DRIVER_LICENSE": "ID_Card",
    "US_PASSPORT": "Passport",
    "shared IN_PAN": "NIN",
}

In [113]:
def get_categories(strings):
    if strings == []:
        return ["Other_data"]
    results = []
    for s in strings:
        category = CATEGORY_MAP.get(s, "Other_data")
        if isinstance(category, list):
            results.extend(category) 
        else:
            results.append(category)  
    return results

def convert_labels(label_str):
    return [label.strip() for label in label_str.replace('{', '').replace('}', '').replace("'", "").split(',')]

def build_dataframe(results_df, threshold_score, threshold_count):
    height, width = results_df.shape
    results_copy = results_df.copy()
    print(f"Threshold score: {threshold_score}\nThreshold count: {threshold_count}")
    empty_row1 = [[] for _ in range(width)]
    empty_row2 = [[] for _ in range(width)]
    empty_row3 = [[] for _ in range(width)]
    results_copy.loc[height] = empty_row1.copy()
    results_copy.loc[height+1] = empty_row2.copy()
    results_copy.loc[height+2] = empty_row3.copy()
    for i in range(width):
        series = pd.Series([item for sublist in results_copy.iloc[2:height:2, i]
                        for item in ast.literal_eval(sublist)], dtype="object").value_counts()
        for a, b in series.items():
            if b > threshold_count:
                results_copy.iloc[height, i].append(a)
                results_copy.iloc[height+1, i].append(b)
    for i in range(width):
        entities = [re.sub(r'_\d+(\.\d+)?$', '', item)
                    for item in results_copy.iloc[height, i] if float(re.search(r'(\d+\.\d+)', item).group(1)) >= threshold_score]
        mapped_entities = get_categories(entities)
        results_copy.iloc[height+2, i] = str(set(mapped_entities))

    results_copy.loc[0] = results_copy.loc[0].apply(convert_labels)
    results_copy.loc[height+2] = results_copy.loc[height+2].apply(convert_labels)
    return results_copy

def compute_performance(results, confusion = False):
    results_copy = results.copy()
    mlb = MultiLabelBinarizer()
    y_true = mlb.fit_transform(results_copy.loc[0])
    y_pred = mlb.transform(results_copy.loc[results_copy.shape[0]-1])
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None, zero_division=0)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    evaluation = pd.DataFrame({
        'Class': mlb.classes_,
        'P': precision,
        'R': recall,
        'F1': f1,
        'S': support
    }).sort_values("S", ascending=False).reset_index(drop=True)
    print(f'Micro F1 Score: {f1_micro:.4f}')
    print(f'Macro F1 Score: {f1_macro:.4f}')
    print(f'Weighted F1 Score: {f1_weighted:.4f}')
    
    if confusion == True:
        y_true_df = pd.DataFrame(y_true)
        y_pred_df = pd.DataFrame(y_pred)

        confusion_matrix = []
        for i in range(20):
            pred_true_compare = y_pred_df.loc[y_pred_df.index.isin(y_true_df.loc[y_true_df[i] == 1].index)]
            arr_label = []
            for j in range(20):
                val = pred_true_compare.loc[:, j].value_counts(normalize=True).get(1)
                arr_label.append(val.item() if val != None else 0)
            confusion_matrix.append(arr_label)
        fig = go.Figure(go.Heatmap(z=confusion_matrix, x=mlb.classes_, y=mlb.classes_, colorscale='Blues'))
        fig.update_layout(title_text='Confusion Matrix', height=800, width=800)
        fig.update_xaxes(title="Predicted Label")
        fig.update_yaxes(title="True Label")
        fig.show()
    return evaluation

# Analyze Presidio Confusion Matrix Part 1

Get a confusion matrix without thresholds to compare all Presidio predictions with the real labels  
Try to find label errors in Dessi  
Use columnwise approach as it is similar to CASSED approach and all three approaches perform similar for no thresholds so the approach does not matter

In [114]:
dessi_cleaned = dessi_all
dessi_labels_cleaned = dessi_all_labels
wrong_columns = []

In [115]:
results_method3_thresholds = build_dataframe(results_method_3_all, 0, 0)
evaluation_method3 = compute_performance(results_method3_thresholds, confusion = True) # Best threshold_score and threshold_count

print(evaluation_method3)

Threshold score: 0
Threshold count: 0
Micro F1 Score: 0.2469
Macro F1 Score: 0.2354
Weighted F1 Score: 0.3321


           Class         P         R        F1     S
0     Other_data  0.137213  0.242951  0.175377  6561
1   Phone_number  0.377567  0.990493  0.546727  4418
2        Address  0.169818  0.843240  0.282704  3802
3         Person  0.204139  0.999721  0.339046  3582
4            NIN  0.219195  0.651773  0.328062  3469
5          Email  1.000000  0.995926  0.997959  3191
6           Date  0.153270  0.985622  0.265286  2782
7            GPE  0.104825  0.911142  0.188020  2172
8   Organization  0.001077  0.001445  0.001234  2076
9    Geolocation  0.035701  0.327502  0.064384  2058
10     SWIFT/BIC  0.000000  0.000000  0.000000   240
11          IBAN  0.892019  0.892019  0.892019   213
12           CCN  0.345098  0.807339  0.483516   109
13      Passport  0.046512  0.388889  0.083086   108
14        Gender  0.000000  0.000000  0.000000    94
15      Religion  0.012819  1.000000  0.025313    93
16     Sexuality  0.000000  0.000000  0.000000    92
17   Nationality  0.012543  1.000000  0.024775

The classes gender, organization, Race, SWIFT/BIC and sexuality can't be predicted by Presidio, as a result P/R/F1 is zero for these classes

Analyse Presidio mistakes  
First focus on the false negatives for every class, so look at the predictions which predicted other classes than the actual groundtruth

In [116]:
def analyse_class(class_label, focus="false negatives"):
    reduced_columns = results_method3_thresholds.iloc[:,dessi_all_labels.loc[dessi_all_labels["label"].str.contains(class_label)].index].T
    indi = []
    if focus == "false negatives":
        for i, a in zip(reduced_columns.index, reduced_columns[204]):
            if class_label not in a:
                indi.append(i)
    elif focus == "true positives":
        for i, a in zip(reduced_columns.index, reduced_columns[204]):
            if class_label in a:
                indi.append(i)
    else:
        raise ValueError("Focus should be either 'false negatives' or 'true positives'")
    d = dessi_all.iloc[:,[int(a) for a in reduced_columns.T[indi].columns.values]]
    add_info = reduced_columns.T[indi].iloc[[0,204],:]
    add_info.columns = d.columns
    df = pd.concat([add_info, d]).reset_index(drop=True)
    return df

'analyse_class' returns the false negative if the parameter 'focus' is not change  
first row display the groundtruth label and the second row of the dataframe the predictions, the other rows are the column values

In [117]:
df_address = analyse_class("Address")
pd.set_option('display.max_columns', 600)
df_address.head()

,aefoxiqjscpw,zip_code,vmwfwqqsnoic,country_code,zip,country_code.1,zip_code.1,zip_code.2,postc,rpxxiumyhlae,zip_code.3,zip_code.4,crxnjlldutum,zip.3,dwecggdbwnco,zipcode.1,pxltxbopsejw,postal_code.2,postal_code.5,jquonjkqmfiq,zip.4,mfeweqkheals,zip_code.6,zipcode.2,postal_code.6,lyydqikplrjv,zip_code.8,country_code.2,eamiqywoifjj,postc.1,zip_code.9,zipcode.3,zip_code.10,jwxpcokehrke,location.1,country_code.4,kilxuqjhkbxy,house_number,zip.6,zip_code.12,postc.2,gchzzbkofkvv,zip.7,kvdojtkwrsss,jkmdosfzqtok,postal_code.10,zip.8,pckfjubhvrso,zip_code.13,zipcode.4,postal_code.11,knoryrzdsoao,zip.9,qxcjviesqxrl,postcode.2,zipcode.5,postcode.3,zip.10,sktqitphfbls,postc.3,zipcode.6,zip_code.15,cbmpaziqnwzr,zip_code.16,zipcode.7,zipcode.8,saahfmplzdnr,postcode.7,zip_code.17,mrvwmvqphvio,zip_code.19,country_code.7,house_number.2,postal_code.14,tcqagfphmhfo,nujfmwqtchow,zipcode.9,zip_code.20,postal_code.15,zipcode.10,oelzemzggkla,cmosuftohfsj,ouhymyfutodh,zip.12,zip.13,postal_code.17,zip_code.21,vxcaynpxyvnc,zipcode.11,postal_code.18,xogafltwrqqc,zip.15,postal_code.19,zip_code.22,sxsymizrrnhf,zip.17,postal_code.20,oifjiggjllae,location_id.5,zip_code.23,jkzemsbeqhbo,zip_code.24,house_number.3,husndpjlsyxe,house_number.4,postc.4,nianjueahqym,house_number.5,postc.5,buijuelmzggf,zipcode.12,zipcode.13,osackobckpuq,quhkldkvphkt,zip_code.27,postcode.10,zipcode.14,house_number.6,ypkdexqrcfcb,zip.19,azjdgelpoejj,postal_code.24,iqzpkozzuzms,djfnkmahehsv,postc.7,zip_code.28,house_number.7,nhibyotdcxqs,zip.20,house_number.8,oeqorpqhverv,vksddewyefhq,zipcode.15,house_number.9,befuvvhxogwi,zip_code.29,rutydrpkxcxx,house_number.10,zlndfnlyrbzd,postc.8,tmvbpmfhhcwv,wswrqbbvsjfr,country_code.14,wxxwbnvuubbf,postc.9,zipcode.16,postcode.11,postal_code.25,zip_code.30,postc.10,house_number.11,xcecpnovxkxd,rufbblgjkafy,nwkohfhvgsfv,teohqlkbbsna,mvxdtldxlurh,country_code.15,zipcode.17,vbrwvaxhvmkg,jtltckqzxslc,zip_code.31,zipcode.18,iaaadyfzqzds,zip_code.32,zip.24,pflmllflggwn,lxbzxczfkrwf,house_number.12,zip_code.34,postc.12,zip_code.35,zip.26,sropxbmzbkbw,zip_code.36,zqwwsgrrcurx,zip_code.37,ssqrqhugrpva,zip_code.38,yigioqblrywx,postcode.12,zip.27,zvlughotfljl,zip_code.39,house_number.13,postcode.13,postc.13,yraplsrtxuwi,postal_code.29,postal_code.30,dtxttdthjbym,house_number.14,house_number.15,house_number.16,zip_code.42,house_number.17,house_number.18,postc.14,zipcode.19,zipcode.20,rdkytiyyfsba,pndocuqrbyuw,city.41,zipcode.21,postc.16,zzdijvkpnaoi,zip_code.43,kucvvtrossuf,mhyhcvwpryrh,zipcode.22,house_number.19,postal_code.34,postc.17,postcode.16,postcode.17,grktyboexsih,postcode.18,zjuvmwfuddoz,postal_code.35,rkfhjyojrqsh,odrrrxajhhez,postal_code.36,zryrhvpuzjml,postc.18,smqbxlhxqiej,zip.29,zipcode.23,zip.30,postc.19,postc.20,country_code.19,house_number.20,fekgmkmctxiy,country_code.20,latitude.32,zip_code.44,xybdpztfbvyq,zip.31,zip_code.45,postal_code.37,lkzjvygxsknf,sniugznsjnjn,zip_code.46,postc.21,xjiaxyqacwgq,gvfiwrgicheh,bccyikuaubbx,postc.22,zip_code.47,yyrxlnmsuzyv,postc.23,zip_code.48,zip.32,tsuoonieoxim,zip.33,postcode.20,jslsaftvdyjv,jfibufhqnhcf,house_number.21,jxyweotltnxt,evvcgfgmbuhv,postal_code.40,zip.34,tkhgvdzeizat,zipcode.25,zipcode.26,postcode.22,eldojcgatook,postcode.23,fapkauysacxt,zip.36,postc.25,country_code.22,zip_code.49,zipcode.27,zip_code.50,zip_code.51,ctupykepdjhj,zip_code.52,zip_code.54,zip_code.55,zipcode.28,zip.37,postal_code.43,zip.38,zip.39,zip.40,country_code.23,zip_code.57,euhfdbmkkgwy,postc.26,goipzzpplgos,zipcode.29,house_number.22,postcode.24,hehofiwojfvo,postc.27,kgurepuevnqv,zip_code.58,zip.42,house_number.23,house_number.24,zip_code.59,country_code.25,postc.28,zip_code.60,postal_code.46,mrxofygsyfip,zipcode.30,wzgweuwnhtfg,postc.29,zip.44,ndiduypjdrbm,zip.45,wktmmsdtqvfi,dejjnairpkhg,kgqlkkwbfeop,postc.30,zipcode.31,uhkirubvlktv,nhbcjglhmrim,euhyjvqtagbw,bfghzbikugme,country_code.29,postal_code.49,iepubepvmfzx,postcode.25,oaergyjxjwxc,zip_code.61,country_code.30,vivzytwkvrys,crgqwwowoaxt,zip_code.62,wzmvsenjjvht,

Presidio can't detect addresses, only Locations (cities, regions, mountains)   
It has problems to detect zip codes, house numbers or region abbreviations     
Cassed even detects house numbers as address

In [118]:
# Filter zip_code and house numbers
indi=[]
for i in range(df_address.shape[1]):
    for s in df_address.iloc[2:-1, i].values:
        s = str(s)
        if bool(re.fullmatch(r"\b(\d{5}|\d{2})\b", s)) or  bool(re.search(r"(?:[a-z].*?){2}", s)):      #check if its five or two digit number or it contains two letters ()
            break
    else:
        indi.append(i)
df_address.iloc[:,indi]

,location.1,location_id.5,city.41
0,[Address],"[Address, Geolocation]","[Address, Geolocation]"
1,[Other_data],"[Phone_number, ID_Card]","[Phone_number, ID_Card]"
2,ROOM 56J,-74.45834,28.356452
3,ROOM 64J,28.953317,28.508385
4,ROOM 11J,28.863564,-75.226822
...,...,...,...
98,ROOM 19J,-74.622737,28.012891
99,ROOM 146J,-75.22466,29.000363
100,ROOM 163J,-74.452212,-74.926844
101,ROOM 194J,-74.402687,28.007002


The column with room still can be an address, but the two lattitudes should only belong to geolocation

In [119]:
dessi_train[df_address.iloc[:,indi[1:]].columns]

,location_id.5,city.41
0,-74.458340,28.356452
1,28.953317,28.508385
2,28.863564,-75.226822
3,39.125028,27.700147
4,-15.471050,28.415501
...,...,...
95,-16.378117,-61.512690
96,-74.622737,28.012891
97,-75.224660,29.000363
98,-74.452212,-74.926844


In [120]:
# find indices of these two columns
indis = []
for e,i in enumerate(dessi_train.columns):
    if i == "location_id.5" or i == "city.41":
        indis.append(e)
dessi_labels_cleaned.loc[indis,:]

,label,split_type
4702,"Address,Geolocation",train
10660,"Address,Geolocation",train


In [121]:
dessi_labels_cleaned.loc[indis,"label"] = "Geolocation"
for ind in indis:
    wrong_columns.append(ind)

Credit Card Number

In [122]:
#Missed to credit card numbers
df_ccn = analyse_class("CCN")
df_ccn.head()

,uarrdsyjqvpz,client vat number,wnzigdnbecdl,voxhfyecngvb,cash card id,smetka nerezidenti.1,nxdqhzmftion,bank card number.3,danbroj,dmhloniyxwtn,alfa1,card number.1,debit card id,id.34,seller account,ebmkwmyqjjbq,card id,payment card number,intf cntpr id,intf ri tp orig code.1,xbygscenrlac
0,[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN],[CCN]
1,"[Date, Other_data]","[Date, Other_data]","[Person, Date, Other_data]","[Date, Other_data]","[Date, Other_data]","[Date, Other_data]","[Date, Other_data]","[Person, Date, Other_data]","[Other_data, GPE, Address, Geolocation, Date]","[Date, Other_data]","[Person, Date, Other_data]","[Date, Other_data]","[Date, Other_data]","[Other_data, GPE, Address, Geolocation, Date]","[Date, Other_data]","[Date, Other_data]","[Person, Date, Other_data]","[Other_data, GPE, Address, Geolocation, Date]","[Other_data, GPE, Address, Geolocation, Date]","[Date, Other_data]","[Date, Other_data]"
2,869935454005056,869978842033190,869954510277112,869917706008401,869917745689492,869937522787046,869909996521938,869992590710644,869999770894155,869906442518633,869926161502386,869955763776577,869936406252655,869948667169262,869920944848042,869917027855688,869996524200721,869983163659805,869908602155784,869919480696260,869905163418684
3,869927810021083,869916829359790,869945050845656,869966368641462,869985659355574,869945156266260,869987133504033,869933511717144,869946237729680,869950855879512,869932730150459,869984885835053,869950640363012,869914040967988,869939128683108,869967753283381,869914356540296,869983338012054,869920518192496,869931903137368,869908224514491
4,869951262014347,869986221162308,869993494327659,869980432148438,869948755528189,869909996521938,869919760677048,869908038761387,869947403973482,869936586342029,869999457534033,869997439269181,869960769888855,869990921797280,869930181943190,869924694553448,869912886830807,869992386049470,869909712494576,869935571040465,869922066186367


In [123]:
def luhn_check(card_number):
    """Check if a credit card number is valid using the Luhn algorithm. Implementation inspired by https://www.geeksforgeeks.org/luhn-algorithm/"""
    card_number = card_number.replace(" ", "")  # Remove spaces if any
    if not card_number.isdigit():
        return False  # Return False if it contains non-digit characters
    
    total = 0
    reverse_digits = card_number[::-1]
    
    for i, digit in enumerate(reverse_digits):
        n = int(digit)
        
        # Double every second digit from the right (1-indexed)
        if i % 2 == 1:
            n *= 2
            if n > 9:
                n -= 9  # Subtract 9 if doubling results in a number > 9
        
        total += n
    
    # Valid if total is a multiple of 10
    return total % 10 == 0

def contains_valid_credit_card(numbers):
    """Check if an array contains at least one valid credit card number."""
    for number in numbers:
        if luhn_check(number):
            return True
    return False


for i in range(df_ccn.shape[1]):
    if not contains_valid_credit_card([str(a) for a in df_ccn.iloc[2:-1,i].values]):
        print(i)
        


All columns contain valid credit card numbers due to the check sum algorithm, it is hard to figure out if presidio considered other options if the credit card is valid or not, presidio is wrong in not detecting them

In [124]:
df_date = analyse_class("Date")
df_date.iloc[[0,1,2,3,-1],:]

,date_account_closed,ncjfynkznere,date_account_opened,yonfgrhhcipw,activefrom.4,date_account_opened.1,hire_date.4,nfzndqloacsd,end_date.1,sdclbhqvzwiq,jlmqizcxwfvs,date_time_activated.12,deleted_date.17,fidyvxbcivtj,cxqucvgmdepk,ishufsvhndaf,txtimwgaxxmm,xqkwyqdhmoyg,jamwjeoijukj,call_start_datetime.13,rccvvsliuwnf,active_to,disconnected_at.2,lastpasswordchangeddate.2,hxtxqprtidjl,deleted_date.12,deleted_date.13,date_started_employment.3,created_date.13,nsodjjjkzgpd,birthday_id,date_employed_to,lastlogindateb,date.4,start_date.3,lastlogindate.1,order_date.1,db_tvr_cls_dt,wvaloveqccdo,hire_date.4
0,[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],"[Date, NIN]",[Date],[Date],"[Date, NIN]",[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date],[Date]
1,[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Person],[Person],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],"[Phone_number, GPE, NIN, Address, Geolocation,...",[Other_data],[Other_data],"[Phone_number, Person, NIN]",[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data]
2,18.05.,08.09.,19.01.,08.04.,14.11.,23.12.,9.05.,24.06.,24.10.,27.05.,8.03.,17.03.,07.03.,10.08.,2.02.,22.08.,20.02.,24.07.,20.05.,22.05.,038 658 225,23.08.,28.05.,057-11-5612,28.12.,26.05.,5.02.,25.06.,08.03.,19.11.,07.11.,26.02.,30.08.,30.12.,02.09.,5.06.,25.12.,11.06.,01.06.,03.05.
3,9.07.,15.04.,8.02.,06.01.,04.12.,24.08.,5.03.,08.03.,25.03.,07.05.,6.12.,21.02.,05.08.,25.01.,4.06.,24.03.,23.08.,28.07.,4.10.,02.02.,660 217 712,24.12.,31.03.,654-41-1027,31.07.,15.07.,16.01.,10.12.,28.10.,07.01.,16.12.,17.11.,22.07.,16.02.,21.01.,16.11.,1.06.,3.10.,06.10.,02.04.
102,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,train,validation,validation,validation,validation,validation,validation,validation,validation,validation,test,test,test,test,test,test,test,test,test,test


Presidio can't detect specific date formats like dd.mm.

In [125]:
for c in ["rccvvsliuwnf", "lastpasswordchangeddate.2"]:
    print(df_date[c].values)

[list(['Date', 'NIN'])
 list(['Phone_number', 'GPE', 'NIN', 'Address', 'Geolocation', 'Organization'])
 '038 658 225' '660 217 712' '176 620 748' '645 116 021' '586 416 208'
 '354 774 606' '250 847 837' '825 861 164' '885 260 265' '522 316 801'
 '752 283 564' '118 347 301' '285 644 605' '315 388 058' '261 054 720'
 '266 580 778' '617 412 374' '658 147 764' '476 670 856' '602 534 521'
 '347 217 200' '317 465 524' '405 465 261' '276 586 724' '380 781 625'
 '527 433 254' '356 277 814' '776 082 356' '285 540 647' '016 216 889'
 '832 670 178' '026 685 776' '347 501 538' '142 762 764' '265 508 622'
 '611 854 464' '538 235 086' '832 852 057' '685 440 166' '265 073 452'
 '220 834 675' '083 582 270' '645 622 101' '665 441 754' '430 881 557'
 '884 520 339' '481 356 202' '016 812 869' '182 254 037' '471 188 060'
 '017 355 447' '260 812 532' '145 168 407' '035 672 146' '180 622 649'
 '674 106 851' '486 104 359' '486 116 809' '220 813 349' '482 035 870'
 '470 433 251' '168 023 729' '108 628 439' '1

In [126]:
# find indices of these two columns
indis = []
for e,i in enumerate(dessi_all.columns):
    if i == "rccvvsliuwnf" or i == "lastpasswordchangeddate.2":
        print(i)
        indis.append(e)
dessi_labels_cleaned.loc[indis,:]

lastpasswordchangeddate.2
rccvvsliuwnf
lastpasswordchangeddate.2
lastpasswordchangeddate.2


,label,split_type
4168,"Date,NIN",train
18592,"Date,NIN",train
20904,"Date,NIN",validation
27102,Other_data,test


In [127]:
indis = [18592, 20904]
dessi_labels_cleaned.loc[indis,"label"] = "NIN"
for ind in indis:
    wrong_columns.append(ind)
dessi_labels_cleaned.loc[indis,:]

,label,split_type
18592,NIN,train
20904,NIN,validation


In [128]:
df_email = analyse_class("Email")
df_email.iloc[[0,1,2,3,4,-1],:]

,email.8,whxwyaaclrhc,email_address_sid.8,city.32,personal_identity_number.19,email_sid.41,email_sid.51,ekszuwwqaaym,brtatrqkunkx,email.13,family_contact_email.24,ssnid.5,email_sid.21
0,"[Email, Person]","[Email, NIN]","[Email, NIN]","[Email, Address]","[Email, NIN]","[Email, Phone_number]","[Email, Phone_number]","[Email, Phone_number]","[Email, Person]","[Email, NIN]","[Email, Phone_number]","[Email, NIN]","[Email, Phone_number]"
1,"[Person, GPE, Nationality, Religion, Address, ...","[Phone_number, Date, NIN]","[Phone_number, Date, Organization, NIN]","[Phone_number, Person, GPE, Nationality, NIN, ...","[Phone_number, Date, NIN]","[Phone_number, Other_data, ID_Card, GPE, NIN, ...","[Phone_number, Other_data, ID_Card, GPE, NIN, ...","[Phone_number, Other_data, Religion, ID_Card, ...","[Person, GPE, Address, Geolocation]","[Phone_number, Other_data, ID_Card, NIN, Date,...","[Phone_number, Other_data, ID_Card, GPE, NIN, ...","[Phone_number, Person, ID_Card, Date]","[Phone_number, Other_data, ID_Card, GPE, NIN, ..."
2,Rodrigo Analise Merritt Felton,698-64-1806,714 083 557,"97 Via Giuseppe Garibaldi, Modigliana, Emilia-...",549-13-6384,146-8631308-7,71800942454,64 38 232 8978,Jairus Seth Knowlton,32146057897,84 6538 4523,ZZ221942T,1399 68837 3875
3,Janja,716-30-1313,508 145 380,"Pailakkatie, Sodankylä, Suomi",465-30-4015,+55 (061) 8999 6405,13165508378,7619608651,Riaan Angelique Meneses,49708352614,(061) 8558-6091,ZZ 526939 T,41 7391-5100
4,Ellison Ike Hall Fortune,354-21-3931,370 528 432,"20 Moulton Avenue, Greenland, New Hampshire 03...",140-11-0700,5124273569,106.378.4050x63511,+ 91 4351 040879,Margo,30469187522,13072658878,ZZ 11 66 63 T,(6482)958 535
102,train,train,train,train,train,train,train,train,test,test,test,test,test


In [129]:
for i in df_email.columns:
    for v in df_email[i].values:
        if "@" in str(v):
            print(i)

In [130]:
indis = []
for e, (i, t) in enumerate(zip(dessi_all.columns, dessi_all.iloc[-1,:])):
    if i in df_email.columns and df_email.loc[102,i] == t:
        print(i)
        indis.append(e)
dessi_labels_cleaned.loc[indis,:]

email.8
whxwyaaclrhc
email_address_sid.8
city.32
personal_identity_number.19
email_sid.41
email_sid.51
ekszuwwqaaym
brtatrqkunkx
email.13
family_contact_email.24
ssnid.5
email_sid.21


,label,split_type
1219,"Email,Person",train
1971,"Email,NIN",train
3609,"Email,NIN",train
7891,"Email,Address",train
9106,"Email,NIN",train
11289,"Email,Phone_number",train
13786,"Email,Phone_number",train
17582,"Email,Phone_number",train
26687,"Email,Person",test
28445,"Email,NIN",test


In [131]:
#None of them contains an Email so all are wrong
for i in [0,8]:
    dessi_labels_cleaned.loc[indis[i],"label"] = "Person"
for i in [1,2,4,9,11]:
    dessi_labels_cleaned.loc[indis[i],"label"] = "NIN"
dessi_labels_cleaned.loc[indis[3],"label"] = "Address"
for i in [5,6,7,10,12]:
    dessi_labels_cleaned.loc[indis[i],"label"] = "Phone_number"
for ind in indis:
    wrong_columns.append(ind)
dessi_labels_cleaned.loc[indis,:]

,label,split_type
1219,Person,train
1971,NIN,train
3609,NIN,train
7891,Address,train
9106,NIN,train
11289,Phone_number,train
13786,Phone_number,train
17582,Phone_number,train
26687,Person,test
28445,NIN,test


**All these columns are wrong labeled there is no email address in there!**

In [132]:
df_gpe = analyse_class("GPE")
df_gpe.iloc[[0,1,2,3,4,102],:]

,state_disctrict,district,vkqsrnnbfnct,placeregion,residence_region,region.2,statedistrict,vhbbuoqdfdwm,htetqgogyoum,regionname,region_name.1,state_or_region.1,district.1,region.3,region.4,residence_region.1,placeregion.1,fkgimpenuxin,saancoqrkvjn,ylpqczfrfjzi,state_or_region.3,partofstate,district.2,state_disctrict.1,czpfcocjvovz,partofstate.1,state_disctrict.2,regionname.1,state_disctrict.3,district.3,partofstate.2,znxnfdhikpam,district.4,state_or_region.4,state_disctrict.4,sclidhkxsgsj,uxedrlhoqwfs,partofstate.3,utibnmzdbbgk,district.5,rwvvpesxwycu,partofstate.4,state_or_region.5,district.6,region.14,partofstate.5,agvnpmgnasti,state_or_region.7,statedistrict.1,state_or_region.8,placeregion.2,partofstate.6,region.19,tucdeiyfguxg,residence_region.2,state_disctrict.5,residence_region.3,placeregion.3,region_name.3,placeregion.4,statedistrict.2,kyfbfndjhrbf,partofstate.7,statedistrict.3,iibapwayertc,state_disctrict.6,state_disctrict.7,region.25,partofstate.8,region.26,regionname.2,regionname.3,region_name.4,partofstate.9,state_or_region.9,state_or_region.10,statedistrict.4,state_disctrict.8,rrqrtqqijblq,cvcycjqeccwh,dvwbsqycdlts,partofstate.10,state_disctrict.9,azvlopulbmbn,region_name.5,region.31,region.32,region.33,ssbgcarxcjzm,partofstate.11,osuhnxlacmif,ysitxblngktq,mwemjsbdbpcc,partofstate.12,region_name.6,oyhgzeobpxmr,placeregion.5,state_disctrict.10,district.7,district.8,partofstate.13,district.9,ughdhomyuivv,region.40,evluxalwfodq,statedistrict.5,district.10,region_name.7,dybxwcdfgovq,vtfopfbqxurc,region.41,statedistrict.6,residence_region.4,zqnbjxksxueq,znmdtpzhbjek,zxleigjdgljj,zcztgkfbpyhd,rvxhnpxjiyjq,statedistrict,regionname,regionname.1,district,district.1,district.2,state_disctrict,partofstate,region_name,state_disctrict.1,region.5,state_or_region.1,pmylvyapjvhg,district.3,ntqwxrbyngnm,residence_region,region.10,region.12,statedistrict.1,region.13,statedistrict.2,partofstate.1,placeregion,bvnblrfyipdt,bqnfntwgndsc,district.4,partofstate.2,region.20,daqjlfabyezv,region.21,rbxdkgouwviz,state_disctrict.2,region.28,placeregion.1,itnrypxxujon,residence_region.1,state_disctrict.3,region,statedistrict,statedistrict.1,region.2,lnwpxxdkmwbk,cymnqgrxqldb,district,residence_region,psdzdiefiuaw,state_disctrict,regionname,ryzfiuutvjcb,state_or_region,district.1,state_disctrict.1,muaefqabibua,district.2,statedistrict.2,state_or_region.2,kvexqumfrago,region_name,statedistrict.3,district.3,xgtnrvnxpybu,state_disctrict.2,state_disctrict.3,yuhgdtdeltot,statedistrict.4,qvqdzjmvesez,district.4,statedistrict.5,region.21,state_disctrict.4,zhjuhnpmwiav,statedistrict.6,tremlgjueqiz,placeregion,state_disctrict.5
0,[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE],[GPE]
1,[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Person],[Other_data],[Other_data],[Person],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Other_data],[Ot

In [133]:
df_gpe = analyse_class("GPE", focus = "true positives")
df_gpe.iloc[[0,1,2,3,4,102],:]

,state,countyname,ywpjaeadqwwj,province,kwwjshqmkqyy,haupfnryhrgq,state.1,postal_code,fzcpjefhgmdt,gpo,at,state_name,postal_code.1,ansi,tiggjgnzixdn,cc3,ygomngppzuda,lgsqmkiqdxhb,dnekjzbhenls,lzabyfqxizwk,name_of_state,sfkfogktoawk,province_title,state_name.1,country_origin,countrycode,gpo.1,ansi.1,uxpphqphrcwv,provincename,gpo.2,qezzlqnunejw,iunpgnipuzgd,ansi.2,countymunicipalityregion,name_of_county,at.1,cc3.1,gpo.3,ansi_abbr,state.3,usps,municipality,county.3,sfihqkwdkgob,cc3.2,ansi_abbr.1,zip_code.5,countrycode.1,postal_code.3,ansi_abbr.2,postal_code.4,province_title.1,iso_code,country_iso,county_municipality_region,iso_code.1,iso,ycztqwzpiqxu,inqttnjgzuoj,country.6,at.2,nxowfjuzcwdn,ykzqwypbhxrs,municipality.1,provincename.1,qbrspwazmskx,iso.1,municipality.2,at.3,at.4,state.4,country_iso.1,province.2,zip_code.7,iso_code.2,wb_region,lnvbmrvxqejs,gipmbavomlkq,postal_code.7,ansi.3,cc3.3,wb_region.1,ansi.4,cc3.4,county.6,lwvssgnppicx,state_or_region,postal_code.8,country.8,country_code.3,cc3.5,county.8,nzjgqccdtllm,dwtwqbbeapcp,state_code,mevatxuxcenl,state.5,province_name,fgiwkydwgrnp,biclikgcugzy,zip_code.11,kchgrgvwqqzc,country_code.5,region_name,state_code.1,znjysnqyssbe,city.6,uwozdftuvisv,cc3.6,state.6,wkhmjaiivrqw,qpogavoprhap,nwmnodjfrece,state.7,ikyowjkgewfm,mdcchelmrejp,iwivpwydaqto,state_name.2,entzgyqrmlhm,state_name.3,country_origin.1,usps.1,iso.2,gpo.4,province.4,bliryjeokamg,xlcvdpiiopxf,jhegtuhkqtmk,placename,province.5,country_name,country_iso.2,cc3.7,cc3.8,smqgmtfpmhgp,gpo.5,iso_code.3,rusvixcepfgn,country_name.1,alneuzlfamyx,countrycode.2,fjiclhznzqon,province.6,wtarqldcfulg,state.8,county.9,ansi_abbr.3,anjawwaqvuaq,country.14,name_of_state.1,ansi_abbr.4,vmcvyjvbbylv,dhwhsjtrpaqi,lzcpxbtumsbk,municipality.5,hszlgepooukq,location.2,lzavtvygakvd,utwkquevjywc,usps.2,cc3.9,wkzoegaunrtv,state_name.4,ygjchphffzgy,postal_code.12,wjlhysslkfax,country_code.6,sqpufsrkibln,state_code.2,usps.3,state.9,lagljrllasyv,country.15,kbllvstkllhj,idiavootfluc,state.11,location.4,tvouguoduddn,country.17,country.18,gpo.6,countrycode.3,tajriqhabrdk,city_name,wsarolyudldo,ubnbojcbzsry,location.5,iso_code.4,rmtuymtuutzk,zip_code.14,country.24,mguylpoftywm,state_name.5,usps.4,sgyrrzprjlzb,at.5,country.26,fjcipzffzyhc,cc3.10,county_municipality_region.1,qiwyfskkpatw,kmzwvcxfzbsj,pygfjdkoycuj,state_code.3,state.12,hwpvouurkyck,ebdeuxsqecif,gpo.7,county.11,zrhldqifmptl,residence_province,dffqmpmmuawg,usps.5,wb_region.2,rdtpglgngqwg,usps.6,usps.7,county_municipality_region.2,origin,xpdckompdjem,country.30,postal_code.13,rqwaqryrkxvr,vtisbywnznma,province_title.2,hxexmtgnacnd,ansi_abbr.5,state_name.6,cc3.11,city.13,zip_code.18,name_of_state.2,residence_county,state_or_region.2,location_of_residence,ansi_abbr.6,trtplmoimyst,municipality.7,country_code.8,residence,iso_code.5,typsktsyedhf,ansi.5,gpo.8,county.12,qcledfhyrljt,cc3.12,mqwknyvxseyn,postal_code.16,municipality.8,icvtlzhmllvg,country_name.2,engcjzrcbkdn,state_name.7,county_municipality_region.3,cc3.13,ansi_abbr.7,country_origin.2,country_origin.3,uscg,state_name.8,state_name.9,name_of_municipality,at.6,vespacwhzccq,fdfgnumhzyba,country.36,yzofbawjgvnk,country_code.9,cc3.14,state.14,ansi_abbr.8,country.37,uyrzbvfefpqi,county_name,county_name.1,state_name.10,state.15,location_of_residence.1,sphfltezfqgh,location_of_residence.2,gpo.9,vwlhwwggjria,gpo.10,name_of_municipality.1,ansi.6,county_municipality_region.4,country_name.3,cc3.15,state.16,cc3.16,municipality.9,ansi.7,state_code.4,qdqrkivzahiu,country.38,county.13,iso.3,zhskpqmazhfz,...,uwmrjrsffdgn,gpo.2,municipality.1,uwnlklvfggig,xihixzlhqjny,state_code.1,name_of_state.1,gsjehpwhahpm,municipality.2,cityname,bbgpgbhbphup,name_of_state.2,wb_region.1,lpzjpwmarxhi,countrycode.2,countymunicipalityregion,location.3,name_of_state.3,zunzvkmmspys,residence_county,country_origin.1,country.8,country_iso.3,yzwczdcesydb,wb_region.2,gpo.3,rpyxrmvqczrb,state_code.2,deycveduudnr,countymunicipalityregion.1,xcengnonfrta,country.9,country.10,origin

**I think Presidio can't detect that, these column values, they are not personal-related**  
 It can only detect cities or county names

In [134]:
df_gpe = analyse_class("GPE")
indis = []
for e, (i, t) in enumerate(zip(dessi_all.columns, dessi_all.iloc[-1,:])):
    try:
        if (i in df_gpe.columns) and (t in df_gpe.loc[102,i].values):
            indis.append(e)
    except:
        if (i in df_gpe.columns) and (t in df_gpe.loc[102,i]):
            indis.append(e)
for ind in indis:
    wrong_columns.append(ind)
    dessi_labels_cleaned.loc[ind,"label"] = "Other_data"

In [135]:
pd.DataFrame(wrong_columns).to_csv("dessi_cleaned/wrong_columns.csv", index=False)
dessi_labels_cleaned.to_csv("dessi_cleaned/all_labels_cleaned.csv", index=False)

Continuation in 'find_label_errors_2.ipynb'